# Analyse de la qualité des données

Préambule

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.rcParams["figure.figsize"] = (12, 8)
plt.rcParams['figure.dpi'] = 100

## Prise de connaissance avec le jeu de données

### Chargement des données brut

In [ ]:
from getting_started import df_patient, df_pcr, pd

df_patient = df_patient.convert_dtypes()
df_pcr = df_pcr.convert_dtypes()

In [ ]:
df_patient.info()

In [ ]:
df_pcr.info()

On définit l'identifiant métier `patient_id` comme index du référentiel.

In [ ]:
df_patient.set_index("patient_id", inplace=True, verify_integrity=True)

La définition de l'index échoue ce qui indique la présence de doublons dans l'identifiant.

Cela justifie une analyse approfondie de la qualité du référentiel.

### Doublons dans l'identifiant

In [ ]:
df_patient[df_patient.duplicated("patient_id", keep=False)].sort_values("patient_id")

Il y a 403 patients qui doublonnent sur l'identifiant. Ils seront supprimés pour le reste de l'analyse.

In [ ]:
df_patient.drop_duplicates("patient_id", keep=False, inplace=True)
df_patient.set_index("patient_id", inplace=True)

df_patient.info()

### Doublons sur l'ensemble des attributs

In [ ]:
df_patient[df_patient.duplicated(keep=False)].sort_values("surname")

Il y 22 patients qui doublonnent sur l'ensemble des attributs, mais qui ont un identifiant différent.

### Attributs non renseignés

In [ ]:
df_patient.notna().sum() / len(df_patient)

Le second champ d'adresse est très peu renseigné (~40%) par rapport aux autres.

In [ ]:
df_patient_isna = df_patient.isna()

df_patient_isna.sum(axis="columns").value_counts().sort_index().plot.bar(xlabel="attributs non renseignés", ylabel="patients", rot=False)

La majorité des patients ont jusqu'à 2 attributs non renseignés.



Distribution des attributs non renseignés pour les cas extêmes (plus de 3 attributs):

In [ ]:
import matplotlib.pyplot as plt
from seaborn import heatmap

plot_kwargs = dict(cmap="Paired", cbar=False, yticklabels=False)

fig, ax = plt.subplots(1, 3)
fig.set_size_inches(20, 6)
heatmap(df_patient_isna[df_patient_isna.sum(axis="columns") == 3], ax=ax[0], **plot_kwargs)
ax[0].set_title("3 attributs non renseignés")
heatmap(df_patient_isna[df_patient_isna.sum(axis="columns") == 4], ax=ax[1], **plot_kwargs)
ax[1].set_title("4 attributs non renseignés")
heatmap(df_patient_isna[df_patient_isna.sum(axis="columns") > 4], ax=ax[2], **plot_kwargs)
ax[2].set_title("Plus de 4 attributs non renseignés")
plt.show()

In [ ]:
df_patient[["surname", "postcode", "phone_number"]].isna().all(axis=1).any()

## Analyse de l'échantillon de tests PCR

### Répartition des valeurs

In [ ]:
df_pcr.pcr.value_counts()

Deux conventions sont utilisées pour représenter les deux valeurs possibles d'un test PCR (négatif ou positif) : `N / P` et `Negative / Positive`.

On préfèrera normaliser ces résultats dans une variable catégorielle.

In [ ]:
df_pcr.pcr = df_pcr.pcr.str[0].astype(pd.CategoricalDtype(categories={"N", "P"}))

df_pcr.pcr.value_counts()

### Exhaustivité du référentiel

Le référentiel de patients n'est pas exhaustif.

In [ ]:
df_pcr.patient_id.drop_duplicates().isin(df_patient.index).all()

Il y aura 168 tests supprimés à l'issue de la jointure avec le référentiel, dont 43 positifs.

In [ ]:
df_pcr[~df_pcr.patient_id.isin(df_patient.index)].pcr.value_counts()

## Analyse du référenciel de patients

### Numéro de téléphone

En [Australie](https://info.australia.gov.au/about-australia/facts-and-figures/telephone-country-and-area-codes), les numéro de téléphone sont composés d'un indicatif régional sur 2 chiffres, suivi d'un indicatif local sur 8 chiffres.

Séparation des indicatifs régionaux et locaux :

In [ ]:
df_patient_phone_number = (
    df_patient.phone_number.dropna()
    .str.extract(r"(\d{2})\s(\d{8})", expand=True)
    .rename(columns={0: "area", 1: "local"})
)

df_patient_phone_number.sample(10, random_state=42)

Vérification de la validité des indicatifs régionaux :

In [ ]:
df_patient_phone_number.area.isin(["02", "03", "04", "07", "08"]).all()

Le numéro de téléphone est fiable, a une forte valeur d'identification et est souvent renseigné.

On s'en servira pour les rapprochements flous.

### Nom et prénom

#### Valeurs manquantes

In [ ]:
df_patient[["surname", "given_name"]].isna().value_counts()

Il y a 861 patients dont le nom et / ou le prénom ne sont pas renseignés.

#### Fautes de saisie

On procède à un rapprochement via le numéro de téléphone et calcule la similarité entre les nom et prénom renseignés.

La mesure de similarité choisie est la distance de Damerau-Levenshtein car elle est particulièrement adaptée pour détecter les fautes humaines qui peuvent arriver lors de la saisie répétée de texte brut, dont l'inversion ou la substitution de caractères.

L'hypothèse est qu'une distance faible corresponde à une faute typographique. Une distance élévée soulèvera d'autres fautes de saisie.

Rapprochement des patients doublonnés sur leur numéro de téléphone

In [ ]:
where_phone_number_duplicated = df_patient.phone_number.duplicated(keep=False)

df = df_patient[where_phone_number_duplicated].dropna(subset={"phone_number"})

df = (
    df[["given_name", "surname", "phone_number"]]
    .dropna(subset={"phone_number"})
    .reset_index()
)

df["full_name"] = df[["given_name", "surname"]].fillna("").sum(axis=1)

df = df.merge(df, on="phone_number").drop_duplicates("patient_id_x")

Calcul de la distance d'édition

In [ ]:
from jellyfish import damerau_levenshtein_distance

df["distance"] = df.apply(lambda x: damerau_levenshtein_distance(x.full_name_x, x.full_name_y), axis=1)

df = df[df.distance >= 1]

Distribution de la similarité

In [ ]:
df.distance.value_counts(normalize=True).sort_index().plot.bar()

Une forte proportion des patients rapprochés ont une faible distance d'édition.

J'évalue la valeur charnière distinguant les fautes typographiques des autres erreurs à 4

In [ ]:
df[df.distance < 4].sample(10, random_state=42)

On retrouve les fautes typographiques classiques:
- Substitution de un ou plusieurs caractères
- Ajout ou suppression de caractères

In [ ]:
df[df.distance == 4].sample(20, random_state=42)

On retrouve un mélange de fautes typographiques plus importantes sur le nom et le prénom, mais aussi des subsitutions de prénom au profit d'une version plus courte ou plus familière (olivia -> livvie, anastasia -> stacia).

In [ ]:
df[df.distance > 4].sample(20, random_state=42)

Pour les fortes distances, on retrouve des inversions de nom et prénom et des noms différents.

## Âge et date de naissance

### Valeurs manquantes

In [ ]:
df_patient[["date_of_birth", "age"]].isna().value_counts()

### Répartition des âges

In [ ]:
df_patient.age.describe()

In [ ]:
df_patient_age = pd.cut(df_patient.age.dropna(), bins=range(0, 101, 10), right=False)

df_patient_age.value_counts().sort_index().plot.bar(xlabel="catégorie d'âge", ylabel="patients")

In [ ]:
df_patient[df_patient.age < 10].age.value_counts().sort_index().plot.bar(xlabel="âge", ylabel="patients")

On remarquera la présence de quelques patients en très bas âge (inférieur à 6 ans).

Je n'ai pas connaissance d'un quelconque âge minimum pour effectuer un test PCR (à confirmer auprès d'un expert).

### Dates de naissance invalides

La date de naissance est stockée au format `YYYYMMDD` dans une valeur entière.

On la convertit en `datetime` afin de la valider.

In [ ]:
date_of_birth = df_patient.date_of_birth.copy()

df_patient.date_of_birth = pd.to_datetime(df_patient.date_of_birth, format="%Y%M%d", errors="coerce")

dob_invalid = date_of_birth[df_patient.date_of_birth.isna()].dropna()

dob_invalid

Il y a 106 patients dont la date de naissance est non conforme.

In [ ]:
dob_invalid_split = (
    dob_invalid.astype(str)
    .str.extract(r"(\d{4})(\d{2})(\d{2})", expand=True)
    .rename(columns={0: "year", 1: "month", 2: "day"})
    .astype(int)
)

dob_invalid_split.sample(10, random_state=42)

L'échantillon montre la présence de valeurs aberrantes pour l'année, le mois et le jour.

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(1, 3)
fig.set_size_inches(20, 6)
dob_invalid_split.year.plot.hist(title="year", ax=ax[0], sharey=True)
dob_invalid_split.month.plot.hist(title="month", ax=ax[1], sharey=True)
dob_invalid_split.day.plot.hist(title="day", ax=ax[2], sharey=True)
plt.show()

### Incohérence entre date de naissance et âge renseigné

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(1, 2)
fig.set_size_inches(20, 6)
df_patient.date_of_birth.dt.year.plot.hist(bins=12, title="year of birth", ax=ax[0], sharey=True)
df_patient.age.plot.hist(bins=12, title="age", ax=ax[1], sharey=True)

plt.show()

Les distributions de l'année de naissance et de l'âge ne concordent pas. La date de naissance est incohérente avec l'âge renseigné.

La distribution de la date de naissance ressemble au résultat d'un tirage aléatoire sur une loi uniforme.

## Adresse personnelle

### Numéro de rue

La numérotation des rues démarrent à 1. Le minimum de 0 est une valeur erronée.

In [ ]:
df_patient.street_number.dropna().astype(int).describe()

Analyse du nombre de chiffres composant le numéro de rue.

In [ ]:
street_number = df_patient.street_number.fillna(0).dropna().astype(int)
street_number["number_of_digits"] = street_number.astype(str).str.len()

street_number.number_of_digits.value_counts().sort_index().plot.bar(xlabel="digits in street number", ylabel="patients")


Il y a quelques patients avec un numéro de rue exceptionnellement élevé (nombre de chiffres >= 5).

On analyse les patients correspondant.

In [ ]:
df_patient_with_large_street_number = df_patient.loc[street_number.number_of_digits >= 5]

df_patient_with_large_street_number[["street_number", "address_1", "phone_number"]]

Les patients ont tous un numéro de téléphone renseigné.

On peut donc effectuer un rapprochement par celui-ci afin d'identifier une possible erreur de saisie.

In [ ]:
df1 = df_patient_with_large_street_number[
    ["street_number", "address_1", "phone_number"]
].reset_index()

df2 = df_patient[
    ["street_number", "address_1", "phone_number"]
].reset_index()

df1 = df1.merge(df2, on="phone_number")

df1.loc[df1.patient_id_x != df1.patient_id_y]

Le numéro de rue à 6 chiffres est une erreur de saisie (342951 -> 3429).

Idem pour l'un des numéro de rue à 5 chiffres (10030 -> 1000).

### Adresse

#### Taux de remplissage

In [ ]:
df_patient[["address_1", "address_2"]].notna().value_counts(normalize=True).sort_index()

- 96% des adresses ont au moins le premier champ d'adresse renseigné
- 38% ont les deux champs renseignés

#### Fautes de saisie

Rapprochement des adresses de patient partageant un même numéro de téléphone.

In [ ]:
address = (
    df_patient[["address_1", "address_2", "phone_number"]]
        .reset_index()
        .dropna(subset=["phone_number"])
        .fillna("<NA>")
)
address["full_address"] = address[["address_1", "address_2"]].apply(", ".join, axis=1)
address = address.merge(address, on="phone_number")
address = address.loc[
    (address.patient_id_x != address.patient_id_y) & \
    (address.full_address_x != address.full_address_y)
]

address.sort_values(by=["phone_number", "patient_id_x"]).head(10)

Calcul de la distance d'édition.

In [ ]:
address["distance"] = address.apply(
    lambda x: damerau_levenshtein_distance(x.full_address_x, x.full_address_y),
    axis=1
)

address.distance.value_counts().sort_index().plot.bar()

Fautes typographiques.

In [ ]:
address.loc[address.distance <= 2, ["full_address_x", "full_address_y", "distance"]].sample(20, random_state=42)

In [ ]:
address.loc[address.distance == 3, ["full_address_x", "full_address_y", "distance"]]

In [ ]:
address.loc[address.distance == 4, ["full_address_x", "full_address_y", "distance"]]

Autres fautes.


In [ ]:
address.loc[address.distance > 4, ["full_address_x", "full_address_y", "distance"]].sample(30, random_state=42)

### Quartier

Certains couples quartier / postcode ont été inversées lors de la saisie.

In [ ]:
df_patient.loc[df_patient.suburb.str.contains(r"\d")]

Pour ces patients, il faut procéder à l'inversion des valeurs du quartier et du code postal.

Le code postal peut contenir un caractère alphabétique qu'il faudra supprimer (exemple `467l0` -> `4670`).

In [ ]:
to_swap = df_patient.loc[df_patient.suburb.str.contains(r"\d"), ["suburb", "postcode"]]

to_swap["suburb"], to_swap["postcode"] = to_swap["postcode"], to_swap["suburb"]

to_swap["postcode"] = to_swap["postcode"].str.replace(r"[a-z]", "")

df_patient.update(to_swap)

df_patient.loc[to_swap.index, ["suburb", "postcode"]]

### État

L'Australie compte 8 états et territoires codés de la manière suivante :

In [ ]:
STATES = {'act', 'nsw', 'nt', 'qld', 'sa', 'tas', 'vic', 'wa'}

Répartition des valeurs de code d'état.

In [ ]:
state_counts = df_patient.state.value_counts()

On retrouve les codes des 8 états de l'Australie.

In [ ]:
state_counts.head(8)

Et d'autres codes d'état à rectifier.

In [ ]:
state_counts[8:]

On effectue un premier rapprochement flou sur une distance d'édition très courte.

Les rapprochements ambigüs (code -> \[état\]) seront supprimés.

In [ ]:
from itertools import product
from jellyfish import damerau_levenshtein_distance

states = STATES
codes = set(state_counts.index) - STATES

df_distance = pd.Series(
    data=[damerau_levenshtein_distance(c, s) for c, s in product(codes, states)],
    index=pd.MultiIndex.from_product([codes, states], names=["code", "state"]),
    name="distance",
)

code_to_state = dict(
    df_distance[df_distance == 1].index.to_frame()
    .drop_duplicates(subset="code", keep=False).index
)

code_to_state

Codes d'état non rapprochés

In [ ]:
codes_to_na = codes - set(code_to_state.keys())

codes_to_na

Correction des états.

In [ ]:
code_to_state.update({c: pd.NA for c in codes_to_na})
df_patient.state.replace(code_to_state, inplace=True)

df_patient.state.value_counts()

### Code postal

#### Taux de remplissage dans l'adresse

In [ ]:
df_patient[["postcode", "suburb", "state"]].notna().value_counts(sort=False)

Le code postal est très souvent renseigné, accompagné du quartier et / ou de l'état.   

#### Validation

Tous les code postaux renseignés respectent le format Australien sur 4 chiffres.

In [ ]:
(~df_patient.dropna().postcode.str.contains(r"\d{4}")).sum()

Chargement du référentiel de codes postaux valides par état.

_Compilé des données de cet [article](https://en.wikipedia.org/w/index.php?title=Postcodes_in_Australia)_

In [ ]:
state_postcode_ranges = pd.read_csv("state_postcode.csv").convert_dtypes()

state_postcode_ranges

Codes postaux invalides hors considération de l'état

In [ ]:
postcodes = df_patient.postcode.dropna().unique()

postcode_range = pd.arrays.IntervalArray.from_arrays(
    left=state_postcode_ranges.postcode_min_range.astype(int),
    right=state_postcode_ranges.postcode_max_range.astype(int),
    closed="both",
)

validate_postcode = lambda p: postcode_range.contains(int(p)).any()

invalid_postcodes = [p for p in postcodes if not validate_postcode(p)]

df_patient.postcode.replace({p: pd.NA for p in invalid_postcodes})

invalid_postcodes

Codes postaux invalides avec considération de l'état

In [ ]:
postcode_ranges_per_state = {
    state: pd.arrays.IntervalArray.from_arrays(
        left=postcode.postcode_min_range,
        right=postcode.postcode_max_range,
        closed="both",
    )
    for state, postcode
    in state_postcode_ranges.groupby(by="state").agg(tuple).iterrows()
}

validate_state_postcode = lambda s, p: postcode_ranges_per_state[s].contains(int(p)).any()

df_patient[["state", "postcode"]].dropna().apply(
    lambda x: not validate_state_postcode(x.state, x.postcode),
    axis="columns"
).value_counts(normalize=True)

Une large proportion de codes postaux sont incohérents avec l'état renseigné (77%).

Il sera nécessaire de corriger l'état renseigné avec le code postal.